# Custom Evaluation Script for MMDetection

The goal of this notebook is to define an eval function based on the competition metric. I wanted to acknowledge [Theo Viel](https://www.kaggle.com/theoviel) for contributions to the helper functions as desribed here: https://www.kaggle.com/theoviel/competition-metric-map-iou and my teammate [KKY](https://www.kaggle.com/evilpsycho42) for the work done for this notebook. 

At a high level, the masks of the ground truths (val set) are compared to the mask predictions (not RLE) over the range of thresholds specified in the competition metric. 

**One pending issue is that the score seems too low. I am hoping that more people taking a look at this can help us root cause the issue.**

## Dependencies

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!rm -rf mmdetection

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .

# %% [code] {"papermill":{"duration":0.077806,"end_time":"2021-10-28T17:40:33.91703","exception":false,"start_time":"2021-10-28T17:40:33.839224","status":"completed"},"tags":[],"execution":{"iopub.status.busy":"2021-11-10T17:41:11.575075Z","iopub.execute_input":"2021-11-10T17:41:11.576605Z","iopub.status.idle":"2021-11-10T17:41:11.583348Z","shell.execute_reply.started":"2021-11-10T17:41:11.576373Z","shell.execute_reply":"2021-11-10T17:41:11.582666Z"}}
%cd ..

In [ ]:
cd /kaggle/working/mmdetection

# **Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cupy as cp
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
import shutil
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed
from mmcv import Config


In [ ]:
%cd ..

# Helper Functions

In [ ]:
def rles_to_mask(encs, shape):
    """
    Decodes a rle.
    Args:
        encs (list of str): Rles for each class.
        shape (tuple [2]): Mask size.
    Returns:
        np array [shape]: Mask.
    """
    img = np.zeros(shape[0] * shape[1], dtype=np.uint)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m
    return img.reshape(shape)


def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.
    Args:
        labels (np array): Labels.
        y_pred (np array): predictions
    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    iou = intersection / union
    
    return iou[1:, 1:]  # exclude background


def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.
    Args:
        threshold (float): Threshold.
        iou (np array [n_truths x n_preds]): IoU matrix.
    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects
    false_positives = np.sum(matches, axis=0) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn


def iou_map(truths, preds, verbose=False):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.
    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.
    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred) for truth, pred in zip(truths, preds)]
    

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)


def combine_mask(masks):
    zeros = np.zeros([520, 704])
    for i, m in enumerate(masks):
        zeros[m] = i+1
    return zeros

def get_mask_from_result(result):
    d = {True : 1, False : 0}
    u,inv = np.unique(result,return_inverse = True)
    mk = np.array([d[x] for x in u])[inv].reshape(result.shape)
    return mk

def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            #print("Overlap detected")
            mask[np.logical_and(mask, other_mask)] = 0
    return mask



In [ ]:
%%writefile ./mmdetection/configs/mask_rcnn/mask_rcnn_r50_fpn_2x_coco_test.py

_base_ = [
    '../_base_/models/mask_rcnn_r50_fpn.py',
    '../_base_/datasets/coco_instance.py',
    '../_base_/schedules/schedule_2x.py', '../_base_/default_runtime.py'
]

# Model paths

In [ ]:
root_dir = '../input/sartorius-cell-instance-segmentation/'
config_base = './mmdetection/configs/'

class CustomConfig():
    config = config_base + 'mask_rcnn/mask_rcnn_r50_fpn_2x_coco.py'
    model_path = '../input/mrcnn-1123-aug-ep30/epoch_30.pth'
    debug = True
    
args = CustomConfig()


# Run Eval 

In [ ]:
val_json = json.load(open("../input/sartorius-cell-instance-segmentation-coco/annotations_val.json", "r"))
cfg = Config.fromfile(args.config)
model_dir = args.model_path

cfg.model.roi_head.bbox_head.num_classes = 3
cfg.model.roi_head.mask_head.num_classes=3

model = init_detector(cfg, model_dir)

confidence_thresholds = {0: 0.5, 1: 0.5, 2: 0.5}

@torch.no_grad()
def calculate_custom_score(model, val_json, root_dir):
    df = pd.read_csv(root_dir + "train.csv")
    val_ids = [i['id'] for i in val_json['images']]
    if args.debug:
        val_ids = val_ids[0:args.debug]
    df_val = df[df.id.isin(val_ids)].reset_index(drop=True)
    df_val = df_val.groupby('id').agg(list).reset_index()
    del df
    for col in df_val.columns[2:]:
        df_val[col] = df_val[col].apply(
            lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x)
        )
    
    gts = []
    dts = []
    for img_id in df_val.id.tolist():
        
        img = mmcv.imread(root_dir + f"train/{img_id}.png")
        result = inference_detector(model, img)
        
        # dt
        dt = []
        for cls, bbs in enumerate(result[0]):
            if bbs.shape != (0, 5):
                sgs = result[1][cls]
                for bb, sg in zip(bbs, sgs):
                    box = bb[:4]
                    cnf = bb[4]
                    if cnf >= confidence_thresholds[cls]:
                        mask = get_mask_from_result(sg)
                        mask = remove_overlapping_pixels(mask, dt)
                        dt.append(mask.astype(bool))
                        #print(mask)
        dts.append(combine_mask(dt))
        
        # gt
        shape = df_val.loc[df_val.id == img_id, ['height', 'width']].values[0]
        gt = rles_to_mask(df_val.loc[df_val.id == img_id, "annotation"].item(), shape).astype(np.uint16)
        gts.append(gt)
    score = iou_map(gts, dts, True)
    return score


score  = calculate_custom_score(model, val_json, root_dir)


### The average precision (AP) seems too low for the model, but it does provide a framework to eval the model for mmdetection. 

In [ ]:
shutil.rmtree('/kaggle/working/mmdetection')